# 7번방의 선물 네이버 영화 리뷰 스크래파이
- https://movie.naver.com/movie/bi/mi/point.naver?code=94775

### 필요 모듈 불러오기

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.chrome.options import Options
import pandas as pd
import os
import numpy as np
import time
from bs4 import BeautifulSoup
import urllib.request as req

In [2]:
# Chrome Browser와 Chrome Driver Version 확인하기
op = Options()
op.add_experimental_option("prefs",{"download.default_directory":r"C:\Users\khdo\Documents\korean_movie\data"})
# chrome_options = webdriver.ChromeOptions()
# selenium ver 4를 사용시 아래코드, 현재 selenium ver은 3으로 본 코드를 사용한다
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver = webdriver.Chrome(ChromeDriverManager().install(), options=op)
driver.get("https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=94775&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1")

[WDM] - Downloading: 100%|██████████| 6.68M/6.68M [00:00<00:00, 33.5MB/s]


In [3]:
# 스크래파이해온 데이터를 넣을 데이터프레임 생성
reviews = pd.DataFrame(columns=["평점", "내용"])
index_num = 0

In [4]:
# 총 리뷰수를 위한 코드
url = driver.current_url
res = req.urlopen(url)
soup = BeautifulSoup(res, "html.parser")

total_review_num = int(soup.select_one("div.score_total > strong > em").string.replace(",",""))

# 총 리뷰수에 따라 반복문을 몇번 돌릴지 결정
if total_review_num % 10 == 0:
    loop_num = total_review_num // 10
else:
    loop_num = total_review_num // 10 + 1
loop_num

4450

In [5]:
for nloop in range(loop_num):

    # 에러 방지 및 기계 아닌척 하기 위한 1초 대기
    time.sleep(1)

    # 현재 드라이버 url을 받아와서 beautifulsoup로 처리
    url = driver.current_url
    res = req.urlopen(url)
    soup = BeautifulSoup(res, "html.parser")

    # 평점에 해당하는 값 받아오기
    score_element = soup.select("div.star_score > em")

    # 리뷰가 길 경우 코드가 달라지기에 긴 리뷰에 대한 인덱싱을 위한 변수
    long_review_num = 0
    for i in range(len(score_element)):
        # 최종 데이터셋에 평점에 값 넣기
        reviews.loc[index_num,"평점"] = int(score_element[i].string)
        try:
            # 최종 데이터셋에 내용에 값 넣기
            reviews.loc[index_num, "내용"] = soup.select_one(f"#_filtered_ment_{i}").string.strip()
        except:
            # 긴 리뷰의 경우 에러가 뜨므로 다른 방식으로 처리
            long_review_element = soup.select("div.score_reple > p > span > span > a")
            reviews.loc[index_num, "내용"] = long_review_element[long_review_num].attrs["data-src"]
            long_review_num = long_review_num + 1
        
        index_num = index_num + 1

    print(f"Page_num : {nloop + 1} complete")
    
    # 다음 페이지 버튼 클릭, 없을 경우 for문 끝내기
    try:
        next_btn = driver.find_element(By.XPATH, f'//*[@id="pagerTagAnchor{nloop+2}"]')
        next_btn.click()
    except:
        break
    

Page_num : 1 complete
Page_num : 2 complete
Page_num : 3 complete
Page_num : 4 complete
Page_num : 5 complete
Page_num : 6 complete
Page_num : 7 complete
Page_num : 8 complete
Page_num : 9 complete
Page_num : 10 complete
Page_num : 11 complete
Page_num : 12 complete
Page_num : 13 complete
Page_num : 14 complete
Page_num : 15 complete
Page_num : 16 complete
Page_num : 17 complete
Page_num : 18 complete
Page_num : 19 complete
Page_num : 20 complete
Page_num : 21 complete
Page_num : 22 complete
Page_num : 23 complete
Page_num : 24 complete
Page_num : 25 complete
Page_num : 26 complete
Page_num : 27 complete
Page_num : 28 complete
Page_num : 29 complete
Page_num : 30 complete
Page_num : 31 complete
Page_num : 32 complete
Page_num : 33 complete
Page_num : 34 complete
Page_num : 35 complete
Page_num : 36 complete
Page_num : 37 complete
Page_num : 38 complete
Page_num : 39 complete
Page_num : 40 complete
Page_num : 41 complete
Page_num : 42 complete
Page_num : 43 complete
Page_num : 44 comple

In [6]:
reviews.to_csv("./data/reviews.csv", index=False)